In [37]:
cd drive/My Drive/app/EGR590

[Errno 2] No such file or directory: 'drive/My Drive/app/EGR590'
/content/drive/My Drive/app/EGR590


In [2]:
import numpy as np
from pathlib import Path
import re
import pandas as pd

%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import keras
import keras.layers as layers
from keras.models import Model
import keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn import metrics

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
df = pd.read_hdf('./training_set.h5', 'balanced')
df

In [5]:
X = df['text'].tolist()
X = np.array(X, dtype=object)[:, np.newaxis]

y = np.asarray(pd.get_dummies(df['label']), dtype = np.int8)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

filename = "./USEmodel"
embed = hub.load(filename)

In [9]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string), axis=[1]))

In [38]:
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding,
	output_shape=(512,))(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
dense = layers.Dense(32, activation='relu')(dense)
pred = layers.Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', 
	optimizer='adam', metrics=['accuracy'])

In [12]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(X_train, 
            y_train,
            validation_data=(X_val, y_val),
            epochs=8,
            batch_size=10)
  model.save_weights('./model.h5')

Train on 2077 samples, validate on 445 samples
Epoch 1/8
2077/2077 [==============================] - 731s 352ms/step - loss: 0.1473 - accuracy: 0.9432 - val_loss: 0.0572 - val_accuracy: 0.9798
Epoch 2/8
2077/2077 [==============================] - 748s 360ms/step - loss: 0.0438 - accuracy: 0.9860 - val_loss: 0.0475 - val_accuracy: 0.9843
Epoch 3/8
2077/2077 [==============================] - 752s 362ms/step - loss: 0.0234 - accuracy: 0.9918 - val_loss: 0.0394 - val_accuracy: 0.9888
Epoch 4/8
2077/2077 [==============================] - 751s 362ms/step - loss: 0.0210 - accuracy: 0.9937 - val_loss: 0.0459 - val_accuracy: 0.9865
Epoch 5/8
2077/2077 [==============================] - 743s 358ms/step - loss: 0.0138 - accuracy: 0.9957 - val_loss: 0.0396 - val_accuracy: 0.9888
Epoch 6/8
2077/2077 [==============================] - 764s 368ms/step - loss: 0.0137 - accuracy: 0.9947 - val_loss: 0.0342 - val_accuracy: 0.9910
Epoch 7/8
2077/2077 [==============================] - 764s 368ms/step 

In [11]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')
  y_preds = model.predict(X_test, batch_size = 10)

In [34]:
y_preds = np.argmax(y_preds, axis=1)
y_test_l = np.argmax(y_test, axis=1)

In [35]:
metrics.confusion_matrix(y_test_l, y_preds)

array([[137,   0],
       [  3, 306]])

In [36]:
print(metrics.classification_report(y_test_l, y_preds))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       137
           1       1.00      0.99      1.00       309

    accuracy                           0.99       446
   macro avg       0.99      1.00      0.99       446
weighted avg       0.99      0.99      0.99       446

